# Facebook & Website & Google Map Mining (Silver 2)

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from Apify_Scrapper_Functions import Get_Phone_Number_From_Facebook, Get_Phone_Number_From_Website

### Import scrapped data

In [2]:
Scrapped_Data = pd.read_parquet("./Staging/Silver/Silver_data_spa.parquet", engine="fastparquet")

# Check if UEN is unique
print(Scrapped_Data["UEN"].is_unique)
print(Scrapped_Data.shape)

True
(6768, 16)


In [3]:
Scrapped_Data["PIC Source 1"] = None
Scrapped_Data["PIC Source 2"] = None
Scrapped_Data["PIC Source 3"] = None
Scrapped_Data = Scrapped_Data[["UEN","Phones", "Emails", "Website", "Facebook", "LinkedIn", "Instagram",  "TikTok", "operational_street", "operational_unit", "operational_postal_code", "operational_address", "PIC Source 1", "PIC Source 2", "PIC Source 3"]]

Scrapped_Data

,UEN,Phones,Emails,Website,Facebook,LinkedIn,Instagram,TikTok,operational_street,operational_unit,operational_postal_code,operational_address,PIC Source 1,PIC Source 2,PIC Source 3
0,202344903C,None,None,https://facewashbearmalaysia.com/,None,None,None,None,30 EASTWOOD ROAD FAIRMOUNT CONDOMINIUM,02-12,486365,30 EASTWOOD ROAD FAIRMOUNT CONDOMINIUM 02-12 S...,None,None,None
1,53401124A,[+6588740083],None,https://landmvnb.com/,[https://www.facebook.com/landmvnb/],None,[https://www.instagram.com/landmvnb/],None,11 IRVING PLACE,01-02,369551,11 IRVING PLACE 01-02 Singapore 369551,None,None,None
2,53448286D,[+6592760927],[joannggg85@gmail.com],None,"[https://m.facebook.com/61551832747678/about/,...",None,None,None,351D ANCHORVALE ROAD ANCHORVALE FIELDS,10-231,544351,351D ANCHORVALE ROAD ANCHORVALE FIELDS 10-231 ...,None,None,None
3,201935987C,[+6587883622],None,https://annielashacademy.com/,[https://www.facebook.com/annieleprolashartist/],None,None,None,8 FRENCH ROAD,13-30,200008,8 FRENCH ROAD 13-30 Singapore 200008,None,None,None
4,202502246E,[+6567329338],[info@zhiholisticbeauty.com],https://www.mycareersfuture.gov.sg/job/custome...,[https://www.facebook.com/zhiholisticbeautycen...,None,[https://www.instagram.com/zhiholisticbeautyce...,None,17 BEDOK SOUTH ROAD,01-569,460017,17 BEDOK SOUTH ROAD 01-569 Singapore 460017,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6763,202223337Z,None,[mailbox@kallos.com.sg],https://www.kallosbeautyco.com/,None,[https://sg.linkedin.com/company/kallos-magazine],[https://www.instagram.com/kallosmag/?hl=en],None,3 ANG MO KIO STREET 62 LINK@AMK,08-02,569139,3 ANG MO KIO STREET 62 LINK@AMK 08-02 Singapor...,None,None,None
6764,201604454R,[+6565625210],None,https://www.mycareersfuture.gov.sg/job/persona...,[https://www.facebook.com/evolution.hub.1/],None,None,None,211 UPPER BUKIT TIMAH ROAD #03- BANYAN TREE HOUSE,03-01,588182,211 UPPER BUKIT TIMAH ROAD #03- BANYAN TREE HO...,None,None,None
6765,202321966E,None,None,None,None,None,None,None,66 TANNERY LANE SINDO INDUSTRIAL BUILDING,01-03L,347805,66 TANNERY LANE SINDO INDUSTRIAL BUILDING 01-0...,None,None,None
6766,202524406M,None,None,None,None,None,None,None,606D TAMPINES STREET 61 TAMPINES GREENRIDGES,14-364,524606,606D TAMPINES STREET 61 TAMPINES GREENRIDGES 1...,None,None,None


### Insert PIC 1 Source that has phone number with RecordOwl

In [4]:
# Assign PIC Source 1 as "RecordOwl" if Phones has any value
Scrapped_Data.loc[Scrapped_Data["Phones"].notna(), "PIC Source 1"] = "RecordOwl"
Scrapped_Data

,UEN,Phones,Emails,Website,Facebook,LinkedIn,Instagram,TikTok,operational_street,operational_unit,operational_postal_code,operational_address,PIC Source 1,PIC Source 2,PIC Source 3
0,202344903C,None,None,https://facewashbearmalaysia.com/,None,None,None,None,30 EASTWOOD ROAD FAIRMOUNT CONDOMINIUM,02-12,486365,30 EASTWOOD ROAD FAIRMOUNT CONDOMINIUM 02-12 S...,None,None,None
1,53401124A,[+6588740083],None,https://landmvnb.com/,[https://www.facebook.com/landmvnb/],None,[https://www.instagram.com/landmvnb/],None,11 IRVING PLACE,01-02,369551,11 IRVING PLACE 01-02 Singapore 369551,RecordOwl,None,None
2,53448286D,[+6592760927],[joannggg85@gmail.com],None,"[https://m.facebook.com/61551832747678/about/,...",None,None,None,351D ANCHORVALE ROAD ANCHORVALE FIELDS,10-231,544351,351D ANCHORVALE ROAD ANCHORVALE FIELDS 10-231 ...,RecordOwl,None,None
3,201935987C,[+6587883622],None,https://annielashacademy.com/,[https://www.facebook.com/annieleprolashartist/],None,None,None,8 FRENCH ROAD,13-30,200008,8 FRENCH ROAD 13-30 Singapore 200008,RecordOwl,None,None
4,202502246E,[+6567329338],[info@zhiholisticbeauty.com],https://www.mycareersfuture.gov.sg/job/custome...,[https://www.facebook.com/zhiholisticbeautycen...,None,[https://www.instagram.com/zhiholisticbeautyce...,None,17 BEDOK SOUTH ROAD,01-569,460017,17 BEDOK SOUTH ROAD 01-569 Singapore 460017,RecordOwl,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6763,202223337Z,None,[mailbox@kallos.com.sg],https://www.kallosbeautyco.com/,None,[https://sg.linkedin.com/company/kallos-magazine],[https://www.instagram.com/kallosmag/?hl=en],None,3 ANG MO KIO STREET 62 LINK@AMK,08-02,569139,3 ANG MO KIO STREET 62 LINK@AMK 08-02 Singapor...,None,None,None
6764,201604454R,[+6565625210],None,https://www.mycareersfuture.gov.sg/job/persona...,[https://www.facebook.com/evolution.hub.1/],None,None,None,211 UPPER BUKIT TIMAH ROAD #03- BANYAN TREE HOUSE,03-01,588182,211 UPPER BUKIT TIMAH ROAD #03- BANYAN TREE HO...,RecordOwl,None,None
6765,202321966E,None,None,None,None,None,None,None,66 TANNERY LANE SINDO INDUSTRIAL BUILDING,01-03L,347805,66 TANNERY LANE SINDO INDUSTRIAL BUILDING 01-0...,None,None,None
6766,202524406M,None,None,None,None,None,None,None,606D TAMPINES STREET 61 TAMPINES GREENRIDGES,14-364,524606,606D TAMPINES STREET 61 TAMPINES GREENRIDGES 1...,None,None,None


### Separate W/O Phone Numbers

In [5]:
# Separate rows with and without phone numbers
df_with_phones = Scrapped_Data[Scrapped_Data["Phones"].notna()]
df_without_phones = Scrapped_Data[Scrapped_Data["Phones"].isna()]

# Store unique phone numbers inside With_Phones_1
df_With_Phones_1 = df_with_phones[df_with_phones["Phones"].duplicated(keep=False) == False]

# Store duplicate phone numbers inside refilter_df_1
df_Duplicate_Phones_1 = df_with_phones[df_with_phones["Phones"].duplicated(keep=False) == True]

# Print shapes
print("Total having Phone numbers:", df_with_phones.shape)
print("Total without Phone numbers:", df_without_phones.shape)
print("Unique phone rows (df_With_Phones_1):", df_With_Phones_1.shape)
print("Duplicate phone rows (df_Duplicate_Phones_1):", df_Duplicate_Phones_1.shape)


Total having Phone numbers: (2769, 15)
Total without Phone numbers: (3999, 15)
Unique phone rows (df_With_Phones_1): (2322, 15)
Duplicate phone rows (df_Duplicate_Phones_1): (447, 15)


### Merge df_without_phones with df_Duplicate_Phones_1

In [6]:
df_without_phones_and_Duplicates = (
    pd.concat([df_Duplicate_Phones_1, df_without_phones], ignore_index=True)
      .drop_duplicates(subset=["UEN"])
)

df_without_phones_and_Duplicates.shape

(4446, 15)

In [7]:
### Final W/O Phones 

print(df_With_Phones_1.shape)
print(df_without_phones_and_Duplicates.shape)

print(df_With_Phones_1["UEN"].is_unique)
print(df_without_phones_and_Duplicates["UEN"].is_unique)

df_without_phones_and_Duplicates["PIC Source 1"] = None

(2322, 15)
(4446, 15)
True
True


### Separate W/O Facebook from df_without_phones_and_Duplicates

In [8]:
# DataFrame with Facebook (not null and not empty)
df_with_facebook = df_without_phones_and_Duplicates[
    df_without_phones_and_Duplicates["Facebook"].notna() &
    (df_without_phones_and_Duplicates["Facebook"] != "")
]

# DataFrame without Facebook (null or empty)
df_without_facebook = df_without_phones_and_Duplicates[
    df_without_phones_and_Duplicates["Facebook"].isna() |
    (df_without_phones_and_Duplicates["Facebook"] == "")
]

# Check the shapes
print(f"With Facebook: {df_with_facebook.shape}")
print(f"Without Facebook: {df_without_facebook.shape}")

With Facebook: (627, 15)
Without Facebook: (3819, 15)


In [9]:
print(df_with_facebook["UEN"].is_unique)


True


In [10]:
df_with_facebook.shape

(627, 15)

### Extract Phone Number of Facebook Pages

In [11]:

facebook_phone_df, facebook_no_phone_df = Get_Phone_Number_From_Facebook(df_with_facebook)


Valid Facebook URLs: 603, Filtered out (profiles/invalid): 24
Processing 603 Facebook pages...
Batch 1/2...
  Original URLs: 500, Unique URLs: 430


[apify.facebook-page-contact-information runId:jnlHNb1vjnKM7st6k] -> Status: RUNNING, Message: 
[apify.facebook-page-contact-information runId:jnlHNb1vjnKM7st6k] -> 2026-01-12T02:53:29.375Z ACTOR: Pulling container image of build SLB5qhar2kFpZIcve from registry.
[apify.facebook-page-contact-information runId:jnlHNb1vjnKM7st6k] -> 2026-01-12T02:53:29.377Z ACTOR: Creating container.
[apify.facebook-page-contact-information runId:jnlHNb1vjnKM7st6k] -> 2026-01-12T02:53:29.423Z ACTOR: Starting container.
[apify.facebook-page-contact-information runId:jnlHNb1vjnKM7st6k] -> 2026-01-12T02:53:29.425Z ACTOR: Running under "LIMITED_PERMISSIONS" permission level.
[apify.facebook-page-contact-information runId:jnlHNb1vjnKM7st6k] -> 2026-01-12T02:53:30.522Z INFO  System info {"apifyVersion":"3.5.1","apifyClientVersion":"2.19.0","crawleeVersion":"3.15.2","osType":"Linux","nodeVersion":"v20.19.6"}
[apify.facebook-page-contact-information runId:jnlHNb1vjnKM7st6k] -> 2026-01-12T02:53:30.755Z INFO  Resul

  Retrieved 418 results from Apify
  Sample item keys: ['facebookUrl', 'categories', 'info', 'likes', 'messenger', 'title', 'address', 'pageId', 'pageName', 'pageUrl']
  DEBUG: URL https://www.facebook.com/p/Lanla-Thai-Ma... | raw_phone=+65 6929 2526
  DEBUG: URL https://www.facebook.com/grandeurspa/... | raw_phone=+65 6310 5589
  DEBUG: URL https://www.facebook.com/RUYIXUANWELLNES... | raw_phone=+65 9353 8706
  DEBUG: URL https://www.facebook.com/beauholic/... | raw_phone=+65 8809 0606
  DEBUG: URL https://www.facebook.com/dbeautyhse/... | raw_phone=+65 6894 6618
  DEBUG: URL https://www.facebook.com/TheBeautyOrigin... | raw_phone=+65 9388 4509
  DEBUG: URL https://www.facebook.com/venusbeauty.sg/... | raw_phone=+65 6777 3936
  DEBUG: URL https://m.facebook.com/newbeautysg/... | raw_phone=+65 6239 5715
  DEBUG: URL https://www.facebook.com/xinyubeauty/... | raw_phone=+65 8787 8858
  DEBUG: URL https://www.facebook.com/p/Pro-Beaute-Sk... | raw_phone=+65 9362 9142
  DEBUG: URL https://w

[apify.facebook-page-contact-information runId:MZWhw3CrR4MgTcD6E] -> Status: RUNNING, Message: 
[apify.facebook-page-contact-information runId:MZWhw3CrR4MgTcD6E] -> 2026-01-12T03:00:18.952Z ACTOR: Pulling container image of build SLB5qhar2kFpZIcve from registry.
[apify.facebook-page-contact-information runId:MZWhw3CrR4MgTcD6E] -> 2026-01-12T03:00:18.953Z ACTOR: Creating container.
[apify.facebook-page-contact-information runId:MZWhw3CrR4MgTcD6E] -> 2026-01-12T03:00:19.172Z ACTOR: Starting container.
[apify.facebook-page-contact-information runId:MZWhw3CrR4MgTcD6E] -> 2026-01-12T03:00:19.173Z ACTOR: Running under "LIMITED_PERMISSIONS" permission level.
[apify.facebook-page-contact-information runId:MZWhw3CrR4MgTcD6E] -> 2026-01-12T03:00:20.427Z INFO  System info {"apifyVersion":"3.5.1","apifyClientVersion":"2.19.0","crawleeVersion":"3.15.2","osType":"Linux","nodeVersion":"v20.19.6"}
[apify.facebook-page-contact-information runId:MZWhw3CrR4MgTcD6E] -> 2026-01-12T03:00:20.812Z INFO  Resul

  Retrieved 103 results from Apify
  Sample item keys: ['url', 'error', 'errorDescription']
  DEBUG: URL https://www.facebook.com/AnagenHairClini... | raw_phone=+91 92222 22407
  DEBUG: URL https://www.facebook.com/TheMorningCatch... | raw_phone=+65 8858 0101
  DEBUG: URL https://www.facebook.com/leehongchuang/p... | raw_phone=None
  DEBUG: URL https://www.facebook.com/evagelicnailsla... | raw_phone=+65 9797 4047
  DEBUG: URL https://www.facebook.com/vybeautystore... | raw_phone=+852 2808 2816
  DEBUG: URL https://m.facebook.com/BeautyMoreSG... | raw_phone=None
  DEBUG: URL https://www.facebook.com/blssantiagoisab... | raw_phone=+63 977 673 7988
  DEBUG: URL https://www.facebook.com/nika.nails7/... | raw_phone=+359 89 677 1174
  DEBUG: URL https://m.facebook.com/p/LV-Massage-1000... | raw_phone=None
  DEBUG: URL https://www.facebook.com/ReginaSingapore... | raw_phone=+65 8781 0969
  DEBUG: URL https://www.facebook.com/pages/Knotty-Af... | raw_phone=None
  DEBUG: URL https://www.faceboo

In [14]:
print("with fb")
print("----------------")
print(facebook_phone_df["Phones"].apply(str).is_unique)
print(facebook_phone_df["UEN"].is_unique)
print(facebook_phone_df.shape)

print("/n without fb")
print("----------------")
print(facebook_no_phone_df["Phones"].apply(str).is_unique)
print(facebook_no_phone_df["UEN"].is_unique)
print(facebook_no_phone_df.shape)

with fb
----------------
True
True
(134, 15)
/n without fb
----------------
False
True
(493, 15)


### Adding Source to facebook_phone_df and merging with df_With_Phones_1


In [15]:
facebook_phone_df

,UEN,Phones,Emails,Website,Facebook,LinkedIn,Instagram,TikTok,operational_street,operational_unit,operational_postal_code,operational_address,PIC Source 1,PIC Source 2,PIC Source 3
3,53429456A,+6593538706,None,https://www.mycareersfuture.gov.sg/job/persona...,https://www.facebook.com/RUYIXUANWELLNESS/,None,[https://www.instagram.com/ru_yi_xuan/?hl=en],[https://tiktok.com/@ruyixuanwellness48],11 JALAN BUKIT MERAH,01-4440,150011,11 JALAN BUKIT MERAH 01-4440 Singapore 150011,None,None,None
4,53386927K,+6588090606,[management@beauholic.com],https://beauholic.wixsite.com/nail,https://www.facebook.com/beauholic/,None,[https://www.instagram.com/beauholic_sg/],None,533 CHOA CHU KANG STREET 51 LIMBANG SHOPPING C...,01-20,680533,533 CHOA CHU KANG STREET 51 LIMBANG SHOPPING C...,None,None,None
8,201827118W,+6568946618,[dsbeauty82@gmail.com],None,https://www.facebook.com/dbeautyhse/,None,None,None,139B LORONG 1A TOA PAYOH THE PEAK @ TOA PAYOH,35-52,312139,139B LORONG 1A TOA PAYOH THE PEAK @ TOA PAYOH ...,None,None,None
10,53478350C,+6567773936,None,https://venusbeauty.com.sg/,https://www.facebook.com/venusbeauty.sg/,None,[https://www.instagram.com/venusbeautysg/?hl=en],None,150 ORCHARD ROAD ORCHARD PLAZA,02-55,238841,150 ORCHARD ROAD ORCHARD PLAZA 02-55 Singapore...,None,None,None
12,202218887Z,+6587878858,[findme@xinyubeauty.com],https://xinyubeauty.com/,https://www.facebook.com/xinyubeauty/,None,[https://www.instagram.com/xinyubeauty2020/],None,142 BEDOK RESERVOIR ROAD EUNOS SPRING,05-1555,470142,142 BEDOK RESERVOIR ROAD EUNOS SPRING 05-1555 ...,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4263,201628451E,+6569693022,None,None,https://www.facebook.com/www.yoyoyo.sg/,None,None,None,5001 BEACH ROAD GOLDEN MILE COMPLEX,04-19,199588,5001 BEACH ROAD GOLDEN MILE COMPLEX 04-19 Sing...,None,None,None
4340,202504877N,+6597626635,None,https://www.liveauracollective.com/,https://www.facebook.com/liveauracollective,None,None,None,160 CHANGI ROAD HEXACUBE,B1-02,419728,160 CHANGI ROAD HEXACUBE B1-02 Singapore 419728,None,None,None
4406,53474670E,+6580781129,None,None,https://www.facebook.com/p/Ping-the-Salon-1000...,None,[https://www.instagram.com/yp_beauty_studio/],None,341 BUKIT BATOK STREET 34,09-52,650341,341 BUKIT BATOK STREET 34 09-52 Singapore 650341,None,None,None
4415,202135974M,+6594510695,None,https://www.mycareersfuture.gov.sg/job/persona...,https://www.facebook.com/beautyshine.singapore/,None,[https://www.instagram.com/beautyshine.sg/?hl=en],None,14 SCOTTS ROAD FAR EAST PLAZA,04-82,228213,14 SCOTTS ROAD FAR EAST PLAZA 04-82 Singapore ...,None,None,None


In [16]:
# Assign PIC Source 1 as "RecordOwl" if Phones has any value
facebook_phone_df.loc[facebook_phone_df["Phones"].notna(), "PIC Source 1"] = "Facebook"


### 

In [17]:
df_With_Phones_2 = pd.concat([df_With_Phones_1, facebook_phone_df], ignore_index=True)


print(df_With_Phones_2["UEN"].is_unique)
print(df_With_Phones_2.shape)

True
(2456, 15)


### Merging back facebook_no_phone_df with df_without_facebook

In [18]:
df_without_phones_2 = df_With_Phones_2 = pd.concat([df_without_facebook, facebook_no_phone_df], ignore_index=True)

print(df_without_phones_2["UEN"].is_unique)
print(df_without_phones_2.shape)

True
(4312, 15)


### Quality Check

In [ ]:
# Check for NA or empty values in Phones and PIC Source 1
print("Phones - NA count:", facebook_phone_df["Phones"].isna().sum())
print("Phones - Empty string count:", (facebook_phone_df["Phones"] == "").sum())
print("PIC Source 1 - NA count:", facebook_phone_df["PIC Source 1"].isna().sum())
print("PIC Source 1 - Empty string count:", (facebook_phone_df["PIC Source 1"] == "").sum())

Phones - NA count: 0
Phones - Empty string count: 0
PIC Source 1 - NA count: 0
PIC Source 1 - Empty string count: 0
Phones - NA count: 3965
Phones - Empty string count: 0
PIC Source 1 - NA count: 4312
PIC Source 1 - Empty string count: 0


In [22]:
# Check where PIC Source 1 is not NA in df_without_phones_2
print("Rows where PIC Source 1 is not NA:")
print(df_without_phones_2[df_without_phones_2["PIC Source 1"].notna()].shape)
print()
df_without_phones_2[df_without_phones_2["PIC Source 1"].notna()]

Rows where PIC Source 1 is not NA:
(0, 15)



,UEN,Phones,Emails,Website,Facebook,LinkedIn,Instagram,TikTok,operational_street,operational_unit,operational_postal_code,operational_address,PIC Source 1,PIC Source 2,PIC Source 3


### Separate W/O Websites

In [23]:
# DataFrame with Websites (not null and not empty)
df_with_websites = df_without_phones_2[
    df_without_phones_2["Website"].notna() &
    (df_without_phones_2["Website"] != "")
]

# DataFrame without Websites (null or empty)
df_without_websites = df_without_phones_2[
    df_without_phones_2["Website"].isna() |
    (df_without_phones_2["Website"] == "")
]

# Check the shapes
print(f"With Websites: {df_with_websites.shape}")
print(f"Without Websites: {df_without_websites.shape}")
print(f"Total from df_without_phones_2: {df_without_phones_2.shape}")

With Websites: (1600, 15)
Without Websites: (2712, 15)
Total from df_without_phones_2: (4312, 15)


### Extract Phone Numbers from Websites

In [24]:
from Apify_Scrapper_Functions import Get_Phone_Number_From_Facebook, Get_Phone_Number_From_Website

scrapped_from_websites = Get_Phone_Number_From_Website(df_with_websites)

🌐 WEBSITE PHONE NUMBER SCRAPER - COST-OPTIMIZED
📊 Configuration:
   • Batch size: 500 websites (increased for efficiency)
   • Concurrency: 3 browsers (parallel processing)
   • Page timeout: 15s (reduced for speed)
   • Function timeout: 30s (optimized)
   • Retries: 1 (minimize compute waste)
   • Browser: Chromium (lightweight)
   • Proxy: RESIDENTIAL (maintained)
   • Strategy: Contact page → Homepage fallback

🔑 Validating Apify API token...
✅ API Key valid - User: SALESOPS_EPOS
   • Plan: STARTER
   • Credits remaining: Check your dashboard at https://console.apify.com/billing

📋 Total rows in RecordOwl_Leads: 1600
📋 Rows with valid websites: 1600

🔍 STEP 1: VERIFYING WEBSITE ACCESSIBILITY
Testing HTTP/HTTPS connectivity for all websites...
  ✅ https://xpressmobilemassage.com/
  ✅ https://www.astutecentre.com.sg/
  ⏭️  https://www.mycareersfuture.gov.sg/job/general-work/beautician-lan-xiang-beauty-massage-e327168adb00732d307a435188b2d867 - Skipped (contains 'mycareersfuture')
  ⏭

[apify.puppeteer-scraper runId:fKWxO7MRVHilpYi8O] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:fKWxO7MRVHilpYi8O] -> 2026-01-12T05:55:16.705Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:fKWxO7MRVHilpYi8O] -> 2026-01-12T05:55:16.706Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:fKWxO7MRVHilpYi8O] -> 2026-01-12T05:55:16.738Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:fKWxO7MRVHilpYi8O] -> 2026-01-12T05:55:16.929Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:fKWxO7MRVHilpYi8O] -> 2026-01-12T05:55:17.563Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:fKWxO7MRVHilpYi8O] -> 2026-01-12T05:55:17.678Z INFO  Configuring Puppeteer Scraper

  ❌ Error during scraping: JSONDecodeError: Expecting value: line 1 column 1 (char 0)
  ❌ Batch error: Error during scraping: JSONDecodeError: Expecting value: line 1 column 1 (char 0)

✅ PROCESSING COMPLETE
📊 Final Statistics:
   • Total rows: 1794
   • Websites verified accessible: 486
   • Websites skipped (keywords): 929
   • Websites scraped: 486
   • Phones found: 0



In [53]:
scrapped_from_websites_copy = scrapped_from_websites.drop_duplicates(subset=["UEN"], keep="first")

# Mask for rows with valid phone values (not NA, not None, not empty string)
mask_has_phone = (
    scrapped_from_websites_copy["Phones"].notna() & 
    (scrapped_from_websites_copy["Phones"].apply(str) != "None") & 
    (scrapped_from_websites_copy["Phones"].apply(str) != "")
)

website_number = scrapped_from_websites_copy[mask_has_phone].copy()
no_website_number = scrapped_from_websites_copy[~mask_has_phone].copy()


In [54]:
# Assign PIC Source 1 as "RecordOwl" if Phones has any value
website_number.loc[website_number["Phones"].notna(), "PIC Source 1"] = "Website"


In [55]:
scrapped_from_websites_copy.shape

(1600, 20)

In [57]:
website_number.shape


(237, 20)

In [58]:
no_website_number.shape

(1363, 20)

### merge with final df

In [59]:
df_With_Phones_1.shape

(2322, 15)

In [60]:
website_number.shape

(237, 20)

In [61]:
df_With_Phones_2 = pd.concat([df_With_Phones_1, website_number], ignore_index=True)

print(df_With_Phones_2["UEN"].is_unique)
print(df_With_Phones_2.shape)

True
(2559, 20)


In [62]:
df_without_websites.shape

(2712, 15)

In [63]:
no_website_number.shape

(1363, 20)

In [64]:
df_without_phones_3 = pd.concat([df_without_websites, no_website_number], ignore_index=True)

print(df_without_phones_3["UEN"].is_unique)
print(df_without_phones_3.shape)

True
(4075, 20)


In [65]:
df_With_Phones_2["UEN"].is_unique

True

In [66]:
df_without_phones_3["UEN"].is_unique

True

### Saving

In [67]:
df_With_Phones_2.to_parquet("./Staging/Gold/spa_with_phone.parquet", index=False, engine="fastparquet")

In [68]:
df_without_phones_3.to_parquet("./Staging/Gold/spa_without_phone.parquet", index=False, engine="fastparquet")